In [19]:
import polars as pl
import pandas as pd

In [26]:
pl.Config.set_thousands_separator(",")
pl.Config.set_tbl_hide_column_data_types(True)
pl.Config.set_float_precision(3)
pl.Config.set_tbl_rows(20)

polars.config.Config

In [27]:
!wget -nc https://ling583.s3.amazonaws.com/books.parquet

File ‘books.parquet’ already there; not retrieving.



In [28]:
c = pl.read_parquet('books.parquet')

In [29]:
c.head()

tok,norm,tag,pos,fileid,sentid
"""[""","""[""","""XX""","""X""","""austen-emma""","""austen-emma_0000"""
"""Emma""","""emma""","""NNP""","""PROPN""","""austen-emma""","""austen-emma_0000"""
"""by""","""by""","""IN""","""ADP""","""austen-emma""","""austen-emma_0000"""
"""Jane""","""jane""","""NNP""","""PROPN""","""austen-emma""","""austen-emma_0000"""
"""Austen""","""austen""","""NNP""","""PROPN""","""austen-emma""","""austen-emma_0000"""


In [30]:
c.with_columns(chars = pl.col('tok').str.len_chars())

tok,norm,tag,pos,fileid,sentid,chars
"""[""","""[""","""XX""","""X""","""austen-emma""","""austen-emma_0000""",1
"""Emma""","""emma""","""NNP""","""PROPN""","""austen-emma""","""austen-emma_0000""",4
"""by""","""by""","""IN""","""ADP""","""austen-emma""","""austen-emma_0000""",2
"""Jane""","""jane""","""NNP""","""PROPN""","""austen-emma""","""austen-emma_0000""",4
"""Austen""","""austen""","""NNP""","""PROPN""","""austen-emma""","""austen-emma_0000""",6
"""1816""","""1816""","""CD""","""NUM""","""austen-emma""","""austen-emma_0000""",4
"""]""","""]""","""-RRB-""","""PUNCT""","""austen-emma""","""austen-emma_0000""",1
"""VOLUME""","""volume""","""NN""","""NOUN""","""austen-emma""","""austen-emma_0000""",6
"""I""","""i""","""PRP""","""PRON""","""austen-emma""","""austen-emma_0000""",1
"""CHAPTER""","""chapter""","""NN""","""NOUN""","""austen-emma""","""austen-emma_0000""",7


In [32]:
c.group_by('fileid').agg(
    n_words = pl.col('tok').count(),
    n_chars = pl.col('tok').str.len_chars().sum(),
    n_sents = pl.col('sentid').n_unique()
).with_columns(
    ari = 4.71 * pl.col('n_chars')/pl.col('n_words') + 0.5 * pl.col('n_words')/pl.col('n_sents') - 21.43
).sort(by='ari')

fileid,n_words,n_chars,n_sents,ari
"""shakespeare-caesar""","25,155","88,972","1,591",3.134
"""shakespeare-hamlet""","36,308","129,405","2,320",3.182
"""shakespeare-macbeth""","22,200","80,188","1,445",3.265
"""burgess-busterbear""","18,739","66,688","1,002",4.683
"""carroll-alice""","34,503","116,009","1,624",5.029
"""bryant-stories""","56,047","194,498","2,719",5.221
"""edgeworth-parents""","209,856","739,177","10,026",5.626
"""chesterton-thursday""","69,927","261,229","3,568",5.965
"""chesterton-ball""","97,936","370,970","4,619",7.012
"""chesterton-brown""","86,456","326,248","3,703",8.017
